In [390]:
import pandas as pd
import numpy as np  # noqa
import optuna
import time
import logging  # noqa
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from lightgbm import LGBMRegressor, early_stopping, log_evaluation

# Enable logging
optuna.logging.set_verbosity(optuna.logging.INFO)


class TimingCallback:
    def __init__(self):
        self.start_time = None

    def __call__(self, study, trial):
        if self.start_time is None:
            self.start_time = time.time()
        else:
            elapsed_time = time.time() - self.start_time
            print(f"Trial {trial.number} finished in {elapsed_time:.2f} seconds.")
            self.start_time = time.time()

In [391]:
# Load data
# data_dir = "/data3/lsf/Pein/Power-Prediction/data/"

train_path = "3-train_merged_55.csv"
test_path = "3-test_merged_55.csv"

train_data_selected = pd.read_csv(train_path)
test_data_selected = pd.read_csv(test_path)

# Set the ['power'] to be non-negative
train_data_selected["power"] = train_data_selected["power"].apply(
    lambda x: x if x >= 0 else 0
)
test_data_selected["power"] = test_data_selected["power"].apply(
    lambda x: x if x >= 0 else 0
)

# Convert 'time' column to datetime
train_data_selected["time"] = pd.to_datetime(train_data_selected["time"])
test_data_selected["time"] = pd.to_datetime(test_data_selected["time"])


# Define features and target
features = [
    col
    for col in train_data_selected.columns
    if col not in ["time", "lead_hour", "initial_time", "power"]
]


def generate_past_features(df, features, seq_len=1, dilation=1):
    """
    Generate past features for a given dataframe.

    Parameters:
        df (pd.DataFrame): The input dataframe.
        features (list): List of features to generate past values for.
        seq_len (int): The length of the sequence (number of past observations).
        dilation (int): The step size between past observations.

    Returns:
        pd.DataFrame: The dataframe with past features, including 'time' and 'power' columns.
    """
    df_copy = df.copy()
    for i in range(1, seq_len):
        for feature in features:
            df_copy[f"{feature}_lag_{i}"] = df_copy[feature].shift(i * dilation)

    # Remove rows with NaN values created due to shifting
    df_copy = df_copy.dropna().reset_index(drop=True)

    return df_copy


# Define your settings
seq_len = 36  # You can change this as needed
dilation = 1  # Default is 1, adjust as needed

# Generate past features
train_data_with_past = generate_past_features(
    train_data_selected, features, seq_len=seq_len, dilation=dilation
)
test_data_with_past = generate_past_features(
    test_data_selected, features, seq_len=seq_len, dilation=dilation
)

# Ensure 'time' and 'power' columns are retained
train_data_with_past = train_data_with_past[
    ["time", "power"] + list(train_data_with_past.columns.difference(["time", "power"]))
]
test_data_with_past = test_data_with_past[
    ["time", "power"] + list(test_data_with_past.columns.difference(["time", "power"]))
]

# Define features and target
features = [
    col
    for col in train_data_with_past.columns
    if col not in ["time", "lead_hour", "initial_time", "power"]
]


/tmp/ipykernel_235734/1652525035.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_copy[f"{feature}_lag_{i}"] = df_copy[feature].shift(i * dilation)
/tmp/ipykernel_235734/1652525035.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_copy[f"{feature}_lag_{i}"] = df_copy[feature].shift(i * dilation)
/tmp/ipykernel_235734/1652525035.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once 

In [392]:
print(features)
print(
    f"shape of train is {train_data_with_past.shape}, shape of test is {test_data_with_past.shape}"
)

['altitude', 'altitude_lag_1', 'altitude_lag_10', 'altitude_lag_11', 'altitude_lag_12', 'altitude_lag_13', 'altitude_lag_14', 'altitude_lag_15', 'altitude_lag_16', 'altitude_lag_17', 'altitude_lag_18', 'altitude_lag_19', 'altitude_lag_2', 'altitude_lag_20', 'altitude_lag_21', 'altitude_lag_22', 'altitude_lag_23', 'altitude_lag_24', 'altitude_lag_25', 'altitude_lag_26', 'altitude_lag_27', 'altitude_lag_28', 'altitude_lag_29', 'altitude_lag_3', 'altitude_lag_30', 'altitude_lag_31', 'altitude_lag_32', 'altitude_lag_33', 'altitude_lag_34', 'altitude_lag_35', 'altitude_lag_4', 'altitude_lag_5', 'altitude_lag_6', 'altitude_lag_7', 'altitude_lag_8', 'altitude_lag_9', 'azimuth', 'azimuth_lag_1', 'azimuth_lag_10', 'azimuth_lag_11', 'azimuth_lag_12', 'azimuth_lag_13', 'azimuth_lag_14', 'azimuth_lag_15', 'azimuth_lag_16', 'azimuth_lag_17', 'azimuth_lag_18', 'azimuth_lag_19', 'azimuth_lag_2', 'azimuth_lag_20', 'azimuth_lag_21', 'azimuth_lag_22', 'azimuth_lag_23', 'azimuth_lag_24', 'azimuth_lag_25'

### Adding time features

In [393]:
# Function to add time features
def add_time_features(df):
    df["time"] = pd.to_datetime(df["time"])

    # Existing time features
    df["hour"] = df["time"].dt.hour
    df["quarter_hour"] = df["time"].dt.minute // 15

    # New time features
    df["day"] = df["time"].dt.day
    df["day_in_week"] = df["time"].dt.weekday

    # Sine and cosine transformations
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

    df["quarter_hour_sin"] = np.sin(2 * np.pi * df["quarter_hour"] / 4)
    df["quarter_hour_cos"] = np.cos(2 * np.pi * df["quarter_hour"] / 4)

    df["day_sin"] = np.sin(2 * np.pi * df["day"] / 31)
    df["day_cos"] = np.cos(2 * np.pi * df["day"] / 31)

    df["day_in_week_sin"] = np.sin(2 * np.pi * df["day_in_week"] / 7)
    df["day_in_week_cos"] = np.cos(2 * np.pi * df["day_in_week"] / 7)

    return df

In [394]:
# # Add time features to both train and test data
train_data_selected = add_time_features(train_data_with_past)
test_data_selected = add_time_features(test_data_with_past)

In [395]:
# Define features including the new time features
time_features = [
    "hour",
    # "quarter_hour",
    "day",
    # "day_in_week",
    "hour_sin",
    "hour_cos",
    # "quarter_hour_sin",
    # "quarter_hour_cos",
    "day_sin",
    "day_cos",
    # "day_in_week_sin",
    # "day_in_week_cos",
]

In [396]:
all_features = features + time_features
y_train = train_data_selected["power"]
y_test = test_data_selected["power"]
train_data_selected = train_data_selected[all_features]
test_data_selected = test_data_selected[all_features]

In [397]:
print(
    f"shape of train is {train_data_selected.shape}, shape of test is {test_data_selected.shape}"
)

shape of train is (14557, 1878), shape of test is (2845, 1878)


In [398]:
print(f"features: {len(features)}, time_features: {len(time_features)}")

features: 1872, time_features: 6


In [399]:
# Extract the new features and target
X_train = train_data_selected
X_test = test_data_selected
print(f"Shape of X_train: {X_train.shape}, Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}, Shape of y_test: {y_test.shape}")


Shape of X_train: (14557, 1878), Shape of y_train: (14557,)
Shape of X_test: (2845, 1878), Shape of y_test: (2845,)


In [400]:
from sklearn.model_selection import train_test_split


def objective(
    trial,
    X_train,
    y_train,
    X_test=None,
    y_test=None,
    use_test_for_validation_flag=False,
    use_sklearn_split=False,
):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 10, 80),
        "max_depth": trial.suggest_int("max_depth", 10, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "n_estimators": trial.suggest_int("n_estimators", 500, 1500),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "num_threads": 16,
        "seed": 42,
    }

    model = LGBMRegressor(**params)

    if use_test_for_validation_flag and X_test is not None and y_test is not None:
        X_val_split, y_val_split = X_test, y_test
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val_split, y_val_split)],
            eval_metric="rmse",
            callbacks=[early_stopping(stopping_rounds=15), log_evaluation(period=500)],
        )
        preds = model.predict(X_val_split)
    else:
        if use_sklearn_split:
            # Use sklearn's train_test_split
            X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
                X_train, y_train, test_size=0.2, random_state=42
            )
        else:
            # Calculate the split index
            split_index = int(0.8 * len(X_train))

            # Split the data manually
            X_train_split, X_val_split = X_train[:split_index], X_train[split_index:]
            y_train_split, y_val_split = y_train[:split_index], y_train[split_index:]

        model.fit(
            X_train_split,
            y_train_split,
            eval_set=[(X_val_split, y_val_split)],
            eval_metric="rmse",
            callbacks=[early_stopping(stopping_rounds=100), log_evaluation(period=500)],
        )
        preds = model.predict(X_val_split)

    # Calculate RMSE
    rmse = root_mean_squared_error(y_val_split, preds)
    return rmse

In [401]:
# Create the study and optimize
use_test_for_validation_flag = False
study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: objective(
        trial,
        X_train,
        y_train,
        X_test,
        y_test,
        use_test_for_validation_flag=use_test_for_validation_flag,
    ),
    n_trials=50,
    callbacks=[TimingCallback()],
)

# Get the best parameters
best_params = study.best_params
print("Best parameters found: ", best_params)

[I 2024-08-18 22:45:40,540] A new study created in memory with name: no-name-55d4572a-eda3-4213-834e-b555846044fc


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.085291 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 477466
[LightGBM] [Info] Number of data points in the train set: 11645, number of used features: 1878
[LightGBM] [Info] Start training from score 105788.593173
Training until validation scores don't improve for 100 rounds


[I 2024-08-18 22:45:45,857] Trial 0 finished with value: 65466.098426212026 and parameters: {'num_leaves': 61, 'max_depth': 26, 'learning_rate': 0.1361168462408431, 'n_estimators': 693, 'min_child_samples': 26, 'subsample': 0.5081834831604161, 'colsample_bytree': 0.6074269705949685}. Best is trial 0 with value: 65466.098426212026.


Early stopping, best iteration is:
[23]	valid_0's rmse: 65466.1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.081209 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 477466
[LightGBM] [Info] Number of data points in the train set: 11645, number of used features: 1878
[LightGBM] [Info] Start training from score 105788.593173
Training until validation scores don't improve for 100 rounds


[I 2024-08-18 22:45:53,326] Trial 1 finished with value: 65850.62002424612 and parameters: {'num_leaves': 49, 'max_depth': 27, 'learning_rate': 0.07053447454989857, 'n_estimators': 1187, 'min_child_samples': 15, 'subsample': 0.48868160862864607, 'colsample_bytree': 0.7154483442352819}. Best is trial 0 with value: 65466.098426212026.


...

Early stopping, best iteration is:
[104]	valid_0's rmse: 66395.9
Trial 47 finished in 12.00 seconds.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.068721 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 477466
[LightGBM] [Info] Number of data points in the train set: 11645, number of used features: 1878
[LightGBM] [Info] Start training from score 105788.593173
Training until validation scores don't improve for 100 rounds


[I 2024-08-18 22:52:12,967] Trial 48 finished with value: 67555.80330552178 and parameters: {'num_leaves': 47, 'max_depth': 19, 'learning_rate': 0.0906547709875118, 'n_estimators': 1277, 'min_child_samples': 14, 'subsample': 0.6201457700855204, 'colsample_bytree': 0.6457477587215775}. Best is trial 11 with value: 65459.4133356878.


Early stopping, best iteration is:
[52]	valid_0's rmse: 67555.8
Trial 48 finished in 5.58 seconds.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.066441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 477466
[LightGBM] [Info] Number of data points in the train set: 11645, number of used features: 1878
[LightGBM] [Info] Start training from score 105788.593173
Training until validation scores don't improve for 100 rounds


[I 2024-08-18 22:52:22,187] Trial 49 finished with value: 67490.91903301542 and parameters: {'num_leaves': 59, 'max_depth': 26, 'learning_rate': 0.0713294213626146, 'n_estimators': 1153, 'min_child_samples': 18, 'subsample': 0.4967251788678449, 'colsample_bytree': 0.8092330290286219}. Best is trial 11 with value: 65459.4133356878.


Early stopping, best iteration is:
[101]	valid_0's rmse: 67490.9
Trial 49 finished in 9.22 seconds.
Best parameters found:  {'num_leaves': 49, 'max_depth': 30, 'learning_rate': 0.07297368356113065, 'n_estimators': 1113, 'min_child_samples': 10, 'subsample': 0.4518093784606565, 'colsample_bytree': 0.6502770948193778}


In [402]:
# best_params = {
#     "num_leaves": 70,
#     "max_depth": 36,
#     "learning_rate": 0.09304350950671668,
#     "n_estimators": 1158,
#     "min_child_samples": 18,
#     "subsample": 0.579731306036922,
#     "colsample_bytree": 0.8511910376678277,
# }

In [403]:
print("Best parameters found: ", best_params)
best_params["num_threads"] = 16

# Train the model with the best parameters
best_model = LGBMRegressor(**best_params)
best_model.fit(X_train, y_train)


# Extract feature importances
feature_importances = best_model.feature_importances_

# Create a DataFrame to hold the features and their importances
importance_df = pd.DataFrame(
    {"Feature": X_train.columns, "Importance": feature_importances}
)

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by="Importance", ascending=False)

Best parameters found:  {'num_leaves': 49, 'max_depth': 30, 'learning_rate': 0.07297368356113065, 'n_estimators': 1113, 'min_child_samples': 10, 'subsample': 0.4518093784606565, 'colsample_bytree': 0.6502770948193778}


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.069748 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 477510
[LightGBM] [Info] Number of data points in the train set: 14557, number of used features: 1878
[LightGBM] [Info] Start training from score 101346.615130


In [404]:
# Predict on the test set
y_pred = best_model.predict(X_test)

# Calculate RMSE
rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE on the test set: {rmse:.4f}")

RMSE on the test set: 63809.0179


In [407]:
# save important_features to csv file
if use_test_for_validation_flag:
    importance_df.to_csv(
        "farm_important_features_use_test_for_validation.csv", index=False
    )
else:
    importance_df.to_csv("farm_important_features.csv", index=False)

In [408]:
feature_importance_df = importance_df
top_K = 5


# Function to extract the base feature name
def extract_base_feature(feature_name):
    if "_lag_" in feature_name:
        return feature_name.split("_lag_")[0]
    return feature_name


# Group features by their base name and sort within each group
grouped_features = feature_importance_df.groupby(
    feature_importance_df["Feature"].apply(extract_base_feature)
)

top_features = []
mean_importances = []

for base_feature, group in grouped_features:
    # Sort the group by importance and select the top_K
    top_group = group.sort_values(by="Importance", ascending=False).head(top_K)

    # Calculate the mean importance for this group
    mean_importance = top_group["Importance"].mean()

    # Add the group to the list with their mean importance
    for _, row in top_group.iterrows():
        top_features.append((row["Feature"], row["Importance"]))

    # Store the base feature and mean importance for sorting later
    mean_importances.append((base_feature, mean_importance))

# Sort the groups by mean importance
sorted_mean_importances = sorted(mean_importances, key=lambda x: x[1], reverse=True)

# Create the final sorted dataframe
final_features = []
for base_feature, _ in sorted_mean_importances:
    for feature, importance in top_features:
        if feature.startswith(base_feature):
            final_features.append((feature, importance))

final_df = pd.DataFrame(final_features, columns=["Feature", "Importance"])

# Display the final dataframe
final_df.to_csv("farm_top_features.csv", index=False)


In [ ]:
# Set the threshold for feature importance
importance_threshold = 0

# Select features with importance greater than or equal to the threshold
selected_features = importance_df[importance_df["Importance"] >= importance_threshold][
    "Feature"
].tolist()

# Separate selected features into original input features and time features
selected_input_features = [
    feature for feature in selected_features if feature not in time_features
]
selected_time_features = [
    feature for feature in selected_features if feature in time_features
]

In [ ]:
# Combine the filtered features with the power column and time features
train_data_final = pd.concat(
    [
        train_data_selected["time"],
        X_train[selected_input_features],
        train_data_selected["power"],
        train_data_selected[time_features],
    ],
    axis=1,
)
test_data_final = pd.concat(
    [
        test_data_selected["time"],
        X_test[selected_input_features],
        test_data_selected["power"],
        test_data_selected[time_features],
    ],
    axis=1,
)

# Ensure the final order of columns
final_columns = ["time"] + selected_input_features + ["power"] + time_features
train_data_final = train_data_final[final_columns]
test_data_final = test_data_final[final_columns]

# Display the new shapes of train_data_final and test_data_final
print("Shape of train_data_final:", train_data_final.shape)
print("Shape of test_data_final:", test_data_final.shape)
print("Selected input features:", selected_input_features)
print(f"Train columns : {train_data_final.columns}")

In [ ]:
# Save the final datasets to CSV
file_name_common = '66_withTime'
train_data_final.to_csv( f"4-train_{file_name_common}.csv", index=False)
test_data_final.to_csv( f"4-test_{file_name_common}.csv", index=False)
print("Final datasets saved to CSV.")